<a href="https://colab.research.google.com/github/gagan3012/nlp-htn/blob/master/Torch_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 

T = torch.Tensor([[1,2],[3,4]])
print(T)

tensor([[1., 2.],
        [3., 4.]])


In [2]:
import numpy as np

N = np.array([[1,2],[3,4]])
print(N)

[[1 2]
 [3 4]]


In [7]:
import torch.nn as nn
import torch.nn.functional as F

x = torch.randn(2,3)
print(x)
x = F.relu(x)
print(x)

tensor([[ 2.1536,  1.1599,  1.0649],
        [ 1.1326,  0.6931, -0.0093]])
tensor([[2.1536, 1.1599, 1.0649],
        [1.1326, 0.6931, 0.0000]])


In [8]:
f = nn.Linear(in_features = 10, out_features = 4)
for n, p in f.named_parameters(): print(n, p.size())

weight torch.Size([4, 10])
bias torch.Size([4])


In [9]:
x = torch.empty(350, 10).normal_()
y = f(x)
print(y.size())

torch.Size([350, 4])


# Model

In [10]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [11]:
import torchvision
import torchvision.transforms as transforms

input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [12]:
model = NeuralNet(input_size, hidden_size, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [17]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28)
        labels = labels
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3045
Epoch [1/5], Step [200/600], Loss: 0.3375
Epoch [1/5], Step [300/600], Loss: 0.1452
Epoch [1/5], Step [400/600], Loss: 0.2273
Epoch [1/5], Step [500/600], Loss: 0.1290
Epoch [1/5], Step [600/600], Loss: 0.1366
Epoch [2/5], Step [100/600], Loss: 0.1156
Epoch [2/5], Step [200/600], Loss: 0.0633
Epoch [2/5], Step [300/600], Loss: 0.0904
Epoch [2/5], Step [400/600], Loss: 0.1032
Epoch [2/5], Step [500/600], Loss: 0.0462
Epoch [2/5], Step [600/600], Loss: 0.0665
Epoch [3/5], Step [100/600], Loss: 0.1000
Epoch [3/5], Step [200/600], Loss: 0.1595
Epoch [3/5], Step [300/600], Loss: 0.0304
Epoch [3/5], Step [400/600], Loss: 0.0157
Epoch [3/5], Step [500/600], Loss: 0.0543
Epoch [3/5], Step [600/600], Loss: 0.0838
Epoch [4/5], Step [100/600], Loss: 0.1031
Epoch [4/5], Step [200/600], Loss: 0.0228
Epoch [4/5], Step [300/600], Loss: 0.0401
Epoch [4/5], Step [400/600], Loss: 0.0669
Epoch [4/5], Step [500/600], Loss: 0.0444
Epoch [4/5], Step [600/600], Loss:

In [18]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the network on the 10000 test images: 97.78 %
